In [3]:
# Importamos todas las librerias necesarias

from bs4 import BeautifulSoup as bs
import requests
import pandas as pd
import numpy as np
import regex as re

In [3]:
url = 'https://agroambient.gva.es/es/precios-agrarios' #Escogemos la url de la pagina web de la Conselleria de Agricultura

response = requests.get(url) # Hacemos una request con esa url a traves de la libreria requests

print(response) # Printeamos para ver si ha sido posible la conexión

html = response.content # Obtenemos la informacion html de la web

<Response [200]>


In [4]:
soup = bs(html, "lxml") # abrimos el contenido html con la libreria Beautiful Soup

li_tags = list(soup.select("li.file a")) # de la pagina html solamente nos interesan los archivos 'a' contenidos dentro de la clase file del archivo 'li'

In [5]:
lista_excel = [] # Cremaos lista de excel vacia

for i in range(len(li_tags)): #iteramos a traves de la variable li_tags 

    if li_tags[i].get("title", "holixls").endswith('xls'): # Nos quedamos únicamente con los archivos acabados en .xls, osea los excel.
        enlaces = li_tags[i]['href'] # de estos archivos nos quedemos con los enlaces a las hojas excel
        lista_excel.append(enlaces) # y las añadimos a la lista excel vacia
    else:
        pass

excel_df = pd.DataFrame(lista_excel, columns = ["media_url"]) # Creamos un dataframe con la lista excel y una columna llamada 'media url'

In [6]:
excel_df["url"]= "https://agroambient.gva.es" + excel_df.media_url[~excel_df.media_url.str.startswith("https")] # añadimos el resto de la url a las obtenidas anteriormente

lista_url_completas = [i for i in excel_df['url'].dropna()] # Hacemos una lista con las url completas y eliminamos los nan

In [7]:
#Cremaos listas vacias equivalentes a los años 2016 hasta el 2022
aniodelsenior2022 = []
aniodelsenior2021 = []
aniodelsenior2020 = []
aniodelsenior2019 = []
aniodelsenior2018 = []
aniodelsenior2017 = []
aniodelsenior2016 = []

for i in range(len(lista_url_completas)): # iteramos sobre la lista de urls completas y añadimos la url de un año determinado a su lista correspondiente creada anteriormente
    if '2022' in lista_url_completas[i]:
        aniodelsenior2022.append(lista_url_completas[i])
    elif '2021' in lista_url_completas[i]:
        aniodelsenior2021.append(lista_url_completas[i])
    elif '2020' in lista_url_completas[i]:
        aniodelsenior2020.append(lista_url_completas[i])
    elif '2019' in lista_url_completas[i]:
        aniodelsenior2019.append(lista_url_completas[i])
    elif '2018' in lista_url_completas[i]:
        aniodelsenior2018.append(lista_url_completas[i])
    elif '2017' in lista_url_completas[i]:
        aniodelsenior2017.append(lista_url_completas[i])
    elif '2016' in lista_url_completas[i]:
        aniodelsenior2016.append(lista_url_completas[i])
    
    else:
        pass

In [8]:
# Creamos dataframes vacios para añadir el contenido de las urls

df_2022 = pd.DataFrame()
df_2021 = pd.DataFrame()
df_2020 = pd.DataFrame()
df_2019 = pd.DataFrame()
df_2018 = pd.DataFrame()
df_2017 = pd.DataFrame()
df_2016 = pd.DataFrame()

In [9]:
def cleaner (aniodelsenior,df): #funcion para limpiar los datasets 
    
    for f in aniodelsenior:
        print(f) # printeamos la url para comprobar que la función esta trabajando
        data = pd.read_excel(f) #leemos el excel a través de la url 
        data1 = data.dropna(how='all').copy() # Eliminamos aquellas columnas que contienen todo nan
        col_data = list(data.columns) # creamos una lista con las columnas del dataframe
        data2 = data1.loc[data1[col_data[0]] != 'PRODUCTO'] # eliminamos de los datos aquellas columnas que se utilizan como header
        data2[[col_data[0], col_data[1], col_data[2]]] = data2[[col_data[0], col_data[1], col_data[2]]].fillna(method='ffill') # Cambiamos los nan por el valor situado inmediatamente por encima
        data1_drop = data2.dropna(thresh=4).loc[:,:col_data[6]] # Eliminamos las filas con menos de 4 valores no nan
        data1_drop2 = data1_drop.loc[:,[col_data[0], col_data[1], col_data[2], col_data[3], col_data[4], col_data[6]]] #cogemos solo las columnas del 0 al 4 y la 6
        data1_drop3 = data1_drop2.rename(columns={col_data[0]: 'producto', col_data[1]:'subtipo', col_data[2]: 'tipo_comercial', col_data[3]:'mercado', col_data[4]:'precio_max', col_data[6]:'precio_min'}) # renombramos columnas
        if 'enero' in f: # para cada url correspondiente a un mes en concreto
            data1_drop3['mes'] = 'enero' # creamos una columna con el nombre de ese mes
            data1_drop3['semana'] = re.findall('Semana\+\d\d',f)[0] # Y otra columna con el nombre de esa semana
            df= df.append(data1_drop3) # Añadimos el dataframe al dataframe vacio creado en la celda anterior
        elif 'febrero' in f:
            data1_drop3['mes'] = 'febrero'
            data1_drop3['semana'] = re.findall('Semana\+\d\d',f)[0]
            df = df.append(data1_drop3)
        elif 'marzo' in f:
            data1_drop3['mes'] = 'marzo'
            data1_drop3['semana'] = re.findall('Semana\+\d\d',f)[0]
            df = df.append(data1_drop3)
        elif 'abril' in f:
            data1_drop3['mes'] = 'abril'
            data1_drop3['semana'] = re.findall('Semana\+\d\d',f)[0]
            df = df.append(data1_drop3)
        elif 'mayo' in f:
            data1_drop3['mes'] = 'mayo'
            data1_drop3['semana'] = re.findall('Semana\+\d\d',f)[0]
            df = df.append(data1_drop3)
        elif 'junio' in f:
            data1_drop3['mes'] = 'junio'
            data1_drop3['semana'] = re.findall('Semana\+\d\d',f)[0]
            df = df.append(data1_drop3)
        elif 'julio' in f:
            data1_drop3['mes'] = 'julio'
            data1_drop3['semana'] = re.findall('Semana\+\d\d',f)[0]
            df = df.append(data1_drop3)
        elif 'agosto' in f:
            data1_drop3['mes'] = 'agosto'
            data1_drop3['semana'] = re.findall('Semana\+\d\d',f)[0]
            df = df.append(data1_drop3)
        elif 'septiembre' in f:
            data1_drop3['mes'] = 'septiembre'
            data1_drop3['semana'] = re.findall('Semana\+\d\d',f)[0]
            df = df.append(data1_drop3)
        elif 'octubre' in f:
            data1_drop3['mes'] = 'octubre'
            data1_drop3['semana'] = re.findall('Semana\+\d\d',f)[0]
            df = df.append(data1_drop3)
        elif 'noviembre' in f:
            data1_drop3['mes'] = 'noviembre'
            data1_drop3['semana'] = re.findall('Semana\+\d\d',f)[0]
            df = df.append(data1_drop3)
        elif 'diciembre' in f:
            data1_drop3['mes'] = 'diciembre'
            data1_drop3['semana'] = re.findall('Semana\+\d\d',f)[0]
            df = df.append(data1_drop3)
        else:
            pass

    return df # la función devuelve el dataframe creado en la celda anterior con toda la información extraida de las url de cada mismo año

In [10]:
def cleaner2 (aniodelsenior,df): # Esta función es muy parecida a la anterior solo cambia una cosa
    
    for f in aniodelsenior:
        print(f)
        data = pd.read_excel(f)
        data1 = data.dropna(how='all').copy()
        col_data = list(data.columns)
        data1[[col_data[0], col_data[1], col_data[2]]] = data1[[col_data[0], col_data[1], col_data[2]]].fillna(method='ffill')
        data1_drop = data1.dropna(thresh=4).loc[:,:col_data[4]]
        data1_drop2 = data1_drop.loc[:,[col_data[0], col_data[1], col_data[2], col_data[3], col_data[4]]]
        data1_drop3 = data1_drop2.rename(columns={col_data[0]: 'producto', col_data[1]:'subtipo', col_data[2]: 'tipo_comercial', col_data[3]:'mercado', col_data[4]:'precio'}) # aqui no hay precio maximo y minimo, ambos estan incluidos en una misma columna con el siguiente formato: preciomin/preciomax
        if 'enero' in f:
            data1_drop3['mes'] = 'enero'
            data1_drop3['semana'] = re.findall('Semana\+\d\d',f)[0]
            df= df.append(data1_drop3)    
        elif 'febrero' in f:
            data1_drop3['mes'] = 'febrero'
            data1_drop3['semana'] = re.findall('Semana\+\d\d',f)[0]
            df = df.append(data1_drop3)
        elif 'marzo' in f:
            data1_drop3['mes'] = 'marzo'
            data1_drop3['semana'] = re.findall('Semana\+\d\d',f)[0]
            df = df.append(data1_drop3)
        elif 'abril' in f:
            data1_drop3['mes'] = 'abril'
            data1_drop3['semana'] = re.findall('Semana\+\d\d',f)[0]
            df = df.append(data1_drop3)
        elif 'mayo' in f:
            data1_drop3['mes'] = 'mayo'
            data1_drop3['semana'] = re.findall('Semana\+\d\d',f)[0]
            df = df.append(data1_drop3)
        elif 'junio' in f:
            data1_drop3['mes'] = 'junio'
            data1_drop3['semana'] = re.findall('Semana\+\d\d',f)[0]
            df = df.append(data1_drop3)
        elif 'julio' in f:
            data1_drop3['mes'] = 'julio'
            data1_drop3['semana'] = re.findall('Semana\+\d\d',f)[0]
            df = df.append(data1_drop3)
        elif 'agosto' in f:
            data1_drop3['mes'] = 'agosto'
            data1_drop3['semana'] = re.findall('Semana\+\d\d',f)[0]
            df = df.append(data1_drop3)
        elif 'septiembre' in f:
            data1_drop3['mes'] = 'septiembre'
            data1_drop3['semana'] = re.findall('Semana\+\d\d',f)[0]
            df = df.append(data1_drop3)
        elif 'octubre' in f:
            data1_drop3['mes'] = 'octubre'
            data1_drop3['semana'] = re.findall('Semana\+\d\d',f)[0]
            df = df.append(data1_drop3)
        elif 'noviembre' in f:
            data1_drop3['mes'] = 'noviembre'
            data1_drop3['semana'] = re.findall('Semana\+\d\d',f)[0]
            df = df.append(data1_drop3)
        elif 'diciembre' in f:
            data1_drop3['mes'] = 'diciembre'
            data1_drop3['semana'] = re.findall('Semana\+\d\d',f)[0]
            df = df.append(data1_drop3)
        else:
            pass

    return df

In [ ]:
df22 = cleaner(aniodelsenior2022, df_2022) #usamos la funcion con la lista de urls del año correspondiente y el dataframe vacio de ese mismo año
df21 = cleaner(aniodelsenior2021, df_2021)


In [ ]:
aniodelsenior2020.remove('https://agroambient.gva.es/documents/162218839/169358588/Semana+49%2C+del+1+al+7+de+diciembre+de+2020+xls.pdf/f95ca99e-a1fd-449a-ad15-90484d7e63cd?t=1608124157193') # eliminamos este archivo que en realidad es un pdf

df20 = cleaner(aniodelsenior2020,df_2020)

In [ ]:
aniodelsenior2019.remove('https://agroambient.gva.es/documents/162218839/167325931/Semana+46%2C+del+12+al+18+de+noviembre+de+2019+xls.xls/ac9fbfc9-31b9-42ff-b86e-e439711e29ad?t=1574250433611') # el archivo excel no funciona

df19 = cleaner(aniodelsenior2019,df_2019)

In [ ]:
aniodelsenior2018_1 = aniodelsenior2018[0:29] # El año 2018 tiene los precios dividos en dos columnas precio max y min pero a partir de la fila 29 los precios maximo y minimo se ponen en la misma columna
df18_1 = cleaner(aniodelsenior2018_1, df_2018)


In [ ]:
aniodelsenior2018_2 = aniodelsenior2018[29:]
df18_2 = cleaner2(aniodelsenior2018_2, df_2018) #Aquí usamos la segunda función de limpieza


In [ ]:
df17 = cleaner2(aniodelsenior2017, df_2017)
df16 = cleaner2(aniodelsenior2016, df_2016)

### Datos de Superficie del REGEPA ###

In [28]:
#Veamos primero cual es la cantidad de naranjas que se plantan en Valencia en relación con otros frutales

regepa17 = pd.read_excel('regepa2017.xls', header=2)

regepa18 = pd.read_excel('regepa2018.xlsx', header=2)

regepa19 = pd.read_excel('regepa2019.xlsx', header=2)

regepa20 = pd.read_excel('regepa2020.xlsx', header=2)

regepa17['año'] = 2017
regepa18['año'] = 2018
regepa19['año'] = 2019
regepa20['año'] = 2020

regepa17 = regepa17.rename(columns={'CULTIVO ':'CULTIVO'})
regepa18 = regepa18.rename(columns={'CULTIVO ':'CULTIVO'})
regepa19 = regepa19.rename(columns={'CULTIVO ':'CULTIVO'})

In [27]:
regepa_total = pd.concat([regepa17,regepa18,regepa19,regepa20]) # Juntamos los datos de todos los años
regepa_total_valencia = regepa_total.loc[regepa_total['PROVINCIA'] == 'VALENCIA', :] # Seleccionamos los de la provincia de Valencia
regepa_frutas_citricos_valencia = regepa_total_valencia.loc[(regepa_total_valencia['GRUPO DE CULTIVO'] == 'CITRICOS') | (regepa_total_valencia['GRUPO DE CULTIVO'] == 'FRUTALES'),:] # Seleccionamos los frutales y los citricos

In [24]:
# Esto lo guardamos como csv en nuestra carpeta
regepa_frutas_citricos_valencia.to_csv(r'C:\Users\Usuario\Desktop\EDA KAQUI\regepa_total4.csv')
